# Recommendation Engine for Adventure Hardware Group

##### Libraries

In [3]:
# libraries importation

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
import pyodbc
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split # for engine evaluation
from sklearn.metrics import mean_squared_error # for engine evaluation


##### Functions

In [5]:
# Establish a connection to my SQL Server database and also deriving a fn to fetch sql table:

def fetch_sql_table(table_name):
    
    conn = pyodbc.connect('Driver={SQL Server};'
                     'Server=OLUWASEUN;'
                      'Database=AdventureWorksDW2022;'
                      'Trusted_Connection=yes;'
                     )
    # Get our Table

    sql_for_df = f'Select FIS.[OrderQuantity] as QuantityOrdered, psc.[EnglishProductSubcategoryName] as ProductSub,FIS.[CustomerKey] as CustomerID, p.[EnglishProductName] as Product,PC.[EnglishProductCategoryName] as ProductCategory from [dbo].[FactInternetSales] as FIS INNER JOIN [dbo].[DimProduct] as P on FIS.[ProductKey] = p.[ProductKey] left join [dbo].[DimProductSubcategory] as psc on psc.[ProductSubcategoryKey] = p.[ProductSubcategoryKey] left join [dbo].[DimProductCategory] as PC on PC.[ProductCategoryKey] = psc.[ProductCategoryKey]' 
                                
    #two parameters: 1. the dataset to extract
    #                2. the connection we created
    df = pd.read_sql(sql_for_df, conn)
    conn.close()
    
    
    return df

# Function to Check for missing values:

def missingValFinder(dataset):
    
    MissingReprt = []
    df = dataset
    cols = df.columns.tolist()
    MissingRecs = df.isnull().sum()
    
    for col, rec in zip(cols, MissingRecs):
        
        if rec > 0:
            MissingReprt.append(f'{col} has {rec} missing records')
        else:
            MissingReprt.append(f'{col} has no missing records')
            
            
    return MissingReprt

In [6]:
# Using the function created above to call Factsinternetsales table

table_name = 'FactInternetSales'

FIS = fetch_sql_table(table_name)

FIS.head()

C:\Users\emmas\AppData\Local\Temp\ipykernel_11356\3791247431.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_for_df, conn)


,QuantityOrdered,ProductSub,CustomerID,Product,ProductCategory
0,1,Road Bikes,21768,"Road-150 Red, 62",Bikes
1,1,Mountain Bikes,28389,"Mountain-100 Silver, 44",Bikes
2,1,Mountain Bikes,25863,"Mountain-100 Silver, 44",Bikes
3,1,Road Bikes,14501,"Road-650 Black, 62",Bikes
4,1,Mountain Bikes,11003,"Mountain-100 Silver, 44",Bikes


In [7]:
FIS.shape

(60398, 5)

In [8]:
# calling our missing value finder function:

missingValFinder(FIS)

['QuantityOrdered has no missing records',
 'ProductSub has no missing records',
 'CustomerID has no missing records',
 'Product has no missing records',
 'ProductCategory has no missing records']

In [9]:
# Dropping columns not needed

col_to_drop = ['Product', 
              'ProductCategory']

df = FIS.drop(col_to_drop, axis=1)

df.head()

,QuantityOrdered,ProductSub,CustomerID
0,1,Road Bikes,21768
1,1,Mountain Bikes,28389
2,1,Mountain Bikes,25863
3,1,Road Bikes,14501
4,1,Mountain Bikes,11003


In [10]:
# df.to_csv('your_file_name.csv', index=False)

In [11]:
df.shape

(60398, 3)

## User based similarity Approach

To build a recommendation engine, we need to create a user-item interaction matrix. This matrix will have CustomerID as rows, ProductCategory as columns, and QuantityOrdered as values.
we'll use a more memory-efficient approach. One option is to use sparse matrices, which store only non-zero elements and save memory.

We will use the csr_matrix from scipy.sparse for efficient storage and computation.

In [14]:
# Create the customer-item interaction matrix
customer_item_matrix = df.pivot_table(index='CustomerID', columns='ProductSub', values='QuantityOrdered', fill_value=0)

# Convert the matrix to a sparse matrix
customer_item_matrix_sparse = csr_matrix(customer_item_matrix.values)


In [15]:
customer_item_matrix

ProductSub,Bike Racks,Bike Stands,Bottles and Cages,Caps,Cleaners,Fenders,Gloves,Helmets,Hydration Packs,Jerseys,Mountain Bikes,Road Bikes,Shorts,Socks,Tires and Tubes,Touring Bikes,Vests
CustomerID,,,,,,,,,,,,,,,,,
11000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
11001,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
11002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
11003,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
11004,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29479,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
29480,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
29481,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
customer_item_matrix.shape


(18484, 17)

In [17]:
customer_item_matrix_sparse

<18484x17 sparse matrix of type '<class 'numpy.float64'>'
	with 44734 stored elements in Compressed Sparse Row format>

In [18]:
# Compute the cosine similarity matrix
customer_similarity_sparse = cosine_similarity(customer_item_matrix_sparse)

# Convert the similarity matrix back to a DataFrame for better readability
customer_similarity_df = pd.DataFrame(customer_similarity_sparse, index=customer_item_matrix.index, columns=customer_item_matrix.index)


In [19]:
customer_similarity_df

CustomerID,11000,11001,11002,11003,11004,11005,11006,11007,11008,11009,...,29474,29475,29476,29477,29478,29479,29480,29481,29482,29483
CustomerID,,,,,,,,,,,,,,,,,,,,,
11000,1.000000,0.617213,0.707107,0.547723,0.816497,0.707107,0.471405,0.912871,0.730297,0.816497,...,0.408248,0.408248,0.408248,0.471405,0.288675,0.408248,0.408248,0.408248,0.408248,0.408248
11001,0.617213,1.000000,0.436436,0.507093,0.566947,0.218218,0.436436,0.507093,0.507093,0.377964,...,0.377964,0.377964,0.377964,0.436436,0.267261,0.377964,0.566947,0.377964,0.377964,0.377964
11002,0.707107,0.436436,1.000000,0.516398,0.866025,0.666667,0.666667,0.774597,0.774597,0.866025,...,0.577350,0.577350,0.577350,0.666667,0.408248,0.577350,0.577350,0.577350,0.577350,0.577350
11003,0.547723,0.507093,0.516398,1.000000,0.447214,0.774597,0.774597,0.600000,0.800000,0.670820,...,0.447214,0.447214,0.447214,0.516398,0.632456,0.447214,0.670820,0.447214,0.447214,0.447214
11004,0.816497,0.566947,0.866025,0.447214,1.000000,0.577350,0.577350,0.894427,0.670820,0.750000,...,0.500000,0.500000,0.500000,0.577350,0.353553,0.500000,0.500000,0.500000,0.500000,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29479,0.408248,0.377964,0.577350,0.447214,0.500000,0.577350,0.577350,0.447214,0.447214,0.500000,...,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000
29480,0.408248,0.566947,0.577350,0.670820,0.500000,0.288675,0.577350,0.447214,0.670820,0.500000,...,0.000000,0.000000,0.000000,0.866025,0.707107,0.000000,1.000000,0.000000,0.000000,0.000000
29481,0.408248,0.377964,0.577350,0.447214,0.500000,0.577350,0.577350,0.447214,0.447214,0.500000,...,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000


In [20]:
# Create a function to recommend customers for a given customer based on customer similarity:


def recommend_similar_customers(CustomerID, customer_similarity_df, num_recommendations=5):
    # Get the similarity scores for all customers with the given customer id
    sim_scores = customer_similarity_df.loc[CustomerID]

    # Sort the customers based on the similarity scores
    sim_scores = sim_scores.sort_values(ascending=False)

    # Get the indices of the most similar customers
    recommended_customers = sim_scores.index[1:num_recommendations+1].tolist()

    return recommended_customers

## Testing the engine:
Generate recommendations for a specific user:

In [22]:
# Specify the customer ID and number of recommendations
CustomerID = 21768  # put one of the customer's id.
num_recommendations = 5

# Generate recommendations
if CustomerID in customer_similarity_df:
    recommended_customers = recommend_similar_customers(CustomerID, customer_similarity_df, num_recommendations)
    print(f"Recommended customers similar to {CustomerID}: {recommended_customers}")

else:
    print(f'{CustomerID} does not exist, please enter a valid customer id')


Recommended customers similar to 21768: [15189, 15915, 11696, 14392, 17334]


## Item based similarity approach

To build a recommendation engine using the item based similarity approach, we need to create a item-customer interaction matrix. This matrix will have ProductSubcategory as rows, CustomerID as columns, and QuantityOrdered as values.
we'll use a more memory-efficient approach. One option is to use sparse matrices, which store only non-zero elements and save memory.

We will use the csr_matrix from scipy.sparse for efficient storage and computation.

In [29]:
# Create the item-user interaction matrix
item_customer_matrix = df.pivot_table(index='ProductSub', columns='CustomerID', values='QuantityOrdered', fill_value=0)

# Convert the matrix to a sparse matrix
item_customer_matrix_sparse = csr_matrix(item_customer_matrix.values)


In [30]:
item_customer_matrix

CustomerID,11000,11001,11002,11003,11004,11005,11006,11007,11008,11009,...,29474,29475,29476,29477,29478,29479,29480,29481,29482,29483
ProductSub,,,,,,,,,,,,,,,,,,,,,
Bike Racks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bike Stands,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bottles and Cages,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
Caps,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
Cleaners,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Fenders,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Gloves,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Helmets,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
Hydration Packs,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
item_customer_matrix.shape

(17, 18484)

In [32]:
# Compute the cosine similarity matrix
item_similarity_sparse = cosine_similarity(item_customer_matrix_sparse)

# Convert the similarity matrix back to a DataFrame for better readability
item_similarity_df = pd.DataFrame(item_similarity_sparse, index=item_customer_matrix.index, columns=item_customer_matrix.index)


In [33]:
item_similarity_df.head()

ProductSub,Bike Racks,Bike Stands,Bottles and Cages,Caps,Cleaners,Fenders,Gloves,Helmets,Hydration Packs,Jerseys,Mountain Bikes,Road Bikes,Shorts,Socks,Tires and Tubes,Touring Bikes,Vests
ProductSub,,,,,,,,,,,,,,,,,
Bike Racks,1.000000,0.024909,0.059222,0.050456,0.071259,0.084531,0.040375,0.045985,0.014481,0.043200,0.078939,0.046467,0.034754,0.025807,0.124014,0.033551,0.023503
Bike Stands,0.024909,1.000000,0.027586,0.037512,0.028193,0.050276,0.041505,0.038224,0.014354,0.038605,0.039125,0.034489,0.018086,0.008140,0.148294,0.023558,0.021745
Bottles and Cages,0.059222,0.027586,1.000000,0.239571,0.158908,0.169153,0.167492,0.279082,0.199080,0.203142,0.288934,0.294780,0.028800,0.079650,0.130675,0.249847,0.115606
Caps,0.050456,0.037512,0.239571,1.000000,0.043197,0.132486,0.050211,0.190201,0.060576,0.264116,0.173069,0.182236,0.171648,0.028396,0.178635,0.193685,0.025694
Cleaners,0.071259,0.028193,0.158908,0.043197,1.000000,0.095675,0.059238,0.054299,0.156334,0.046672,0.093047,0.090453,0.037066,0.024307,0.198123,0.064264,0.021486


In [34]:
item_similarity_df.shape

(17, 17)

In [35]:
# Create a function to recommend items for a given item based on item similarity:


def recommend_similar_items(item_id, item_similarity_df, num_recommendations=5):
    # Get the similarity scores for all items with the given item
    sim_scores = item_similarity_df.loc[item_id]

    # Sort the items based on the similarity scores
    sim_scores = sim_scores.sort_values(ascending=False)

    # Get the indices of the most similar items
    recommended_items = sim_scores.index[1:num_recommendations+1].tolist()

    return recommended_items


### Testing the system by recommending items for several products.

In [37]:
# Specify the item ID and number of recommendations
item_id = 'Bike Racks'  # put one of the items in the product sub category
num_recommendations = 5

# Generate recommendations
if item_id in item_similarity_df.index:
    recommended_items = recommend_similar_items(item_id, item_similarity_df, num_recommendations)
    print(f"Recommended items similar to {item_id}: {recommended_items}")
else:
    print(f'{item_id} does not exist, please input a valid product')


Recommended items similar to Bike Racks: ['Tires and Tubes', 'Fenders', 'Mountain Bikes', 'Cleaners', 'Bottles and Cages']


# Popularity based Approach

In [44]:
# Create the customer-item interaction matrix
customer_item_matrix1 = df.pivot_table(index='CustomerID', columns='ProductSub', values='QuantityOrdered', fill_value=0)



In [45]:
# Convert the matrix to a sparse matrix
customer_item_matrix_sparse1 = csr_matrix(customer_item_matrix1.values)

In [46]:
customer_item_matrix1

ProductSub,Bike Racks,Bike Stands,Bottles and Cages,Caps,Cleaners,Fenders,Gloves,Helmets,Hydration Packs,Jerseys,Mountain Bikes,Road Bikes,Shorts,Socks,Tires and Tubes,Touring Bikes,Vests
CustomerID,,,,,,,,,,,,,,,,,
11000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
11001,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
11002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
11003,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
11004,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29479,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
29480,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
29481,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
# Compute the cosine similarity matrix
customer_similarity_sparse1 = cosine_similarity(customer_item_matrix_sparse1)



In [48]:
customer_similarity_sparse1.shape

(18484, 18484)

In [49]:
# Convert the similarity matrix back to a DataFrame for better readability
customer_similarity_df1 = pd.DataFrame(customer_similarity_sparse1, index=customer_item_matrix.index, columns=customer_item_matrix.index)

In [50]:
def recommend_items(customer_id, customer_similarity_sparse1, customer_item_matrix1, num_recommendations=5):
    # Get the index of the user
    customer_index = customer_item_matrix1.index.get_loc(customer_id)

    # Get the similarity scores for all customers with the given customer
    sim_scores = customer_similarity_df1.iloc[customer_index]

    # Sort the products based on the similarity scores
    sim_scores = sim_scores.sort_values(ascending=False)

    # Get the indices of the most similar products
    sim_indices = sim_scores.index[1:]

    # Get the items interacted by similar users
    similar_customers_items = customer_item_matrix1.loc[sim_indices].sum(axis=0)
    
    # Remove items already interacted by the user
    customer_items = customer_item_matrix1.loc[customer_id]
    similar_customers_items = similar_customers_items[customer_items == 0]

    # Get the top recommended items
    recommended_items = similar_customers_items.sort_values(ascending=False).index[:num_recommendations].tolist()

    return recommended_items


### Testing the engine

In [52]:
# Specify the customer ID and number of recommendations
customer_id = 14501 # put one of the customer's id.
num_recommendations = 5

# Generate recommendations
if customer_id in customer_similarity_df1:
   recommended_items = recommend_items(customer_id, customer_similarity_df1, customer_item_matrix1, num_recommendations)
   print(f"Recommended items for customer {customer_id}: {recommended_items}")

else:
    print(f'{Customer_id} does not exist, please enter a valid customer id')

Recommended items for customer 14501: ['Tires and Tubes', 'Helmets', 'Bottles and Cages', 'Jerseys', 'Touring Bikes']
